<strong><center><h1>Analyzing Stock Options with Machine Learning</h1></center></strong>

## **1. Introduction**

#### For many years, traders and merchants developed and practiced techniques involving the prediction of prices for goods. These techniques were then used for predicting stock prices (Hayes, 2019). Many of these techniques are still used today. Recently, more traders have been relying on the use of algorithms for automated trading and price prediction. An advantage many have found with using algorithms is that it strips trading of personal attachment (i.e. emotions). This project in focuses more on the prediction of stock options.

#### Stock options are contracts that give an individual the right to **buy** or **sell** a stock at a specific price by a specific date (Palmer, 2019). They can either be a buy or a sell contract known as a **call** and a **put**, respectively. These options allow an individual to buy or sell their option if the price of a stock reaches a certain price point. This project involves the use of historical stock option data to predict a call or a put for future option investments.

#### The goal of this project is to make observations of multiple models involved with predicting stock options and compare them.

## **2. Data**

### **A. Importing Libraries and Data**

In [1]:
from copy import deepcopy as copy
from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns

#### There are multiple sources of data located in the **Sample_L2_2018** directory. The name of the dataset correlates to when the data was collected (i.e. **20180102** = **01/02/2018**).


#### For the initial experiments, we will use the data **L2_options_20180102.csv**. 

In [2]:
options = pd.read_csv("Sample_L2_2018/L2_options_20180102.csv")
options

,UnderlyingSymbol,UnderlyingPrice,Exchange,OptionSymbol,OptionExt,Type,Expiration,DataDate,Strike,Last,Bid,Ask,Volume,OpenInterest,IV,Delta,Gamma,Theta,Vega,AKA
0,NFLX,201.07,*,NFLX180105C00140000,NaN,call,01/05/2018,01/02/2018,140.0,0.00,60.65,61.70,0,0,0.4771,1.0000,0.0000,-2.2535,0.0000,NFLX180105C00140000
1,NFLX,201.07,*,NFLX180105P00140000,NaN,put,01/05/2018,01/02/2018,140.0,0.01,0.00,0.03,33,0,0.5949,0.0000,0.0000,0.0000,0.0000,NFLX180105P00140000
2,NFLX,201.07,*,NFLX180105C00145000,NaN,call,01/05/2018,01/02/2018,145.0,0.00,55.55,56.70,0,0,0.4771,1.0000,0.0000,-2.3340,0.0000,NFLX180105C00145000
3,NFLX,201.07,*,NFLX180105P00145000,NaN,put,01/05/2018,01/02/2018,145.0,0.00,0.00,0.03,0,0,0.5949,0.0000,0.0000,0.0000,0.0000,NFLX180105P00145000
4,NFLX,201.07,*,NFLX180105C00150000,NaN,call,01/05/2018,01/02/2018,150.0,37.00,50.85,51.70,0,9,0.4771,1.0000,0.0000,-2.4145,0.0000,NFLX180105C00150000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,NFLX,201.07,*,NFLX200117P00270000,NaN,put,01/17/2020,01/02/2018,270.0,0.00,79.05,83.50,0,0,0.3462,-0.6113,0.0039,-6.0559,110.0646,NFLX200117P00270000
1080,NFLX,201.07,*,NFLX200117C00280000,NaN,call,01/17/2020,01/02/2018,280.0,19.15,17.00,21.50,6,340,0.3482,0.3627,0.0038,-10.0582,107.6948,NFLX200117C00280000
1081,NFLX,201.07,*,NFLX200117P00280000,NaN,put,01/17/2020,01/02/2018,280.0,0.00,86.50,91.00,0,0,0.3443,-0.6409,0.0038,-5.5576,107.3281,NFLX200117P00280000
1082,NFLX,201.07,*,NFLX200117C00290000,NaN,call,01/17/2020,01/02/2018,290.0,16.70,14.95,17.25,792,181,0.3361,0.3247,0.0037,-9.3033,103.3016,NFLX200117C00290000


#### In order to help determine relevant features, we will call the <code>unique()</code> method provided by the pandas library in order to check unique values in a feature.

#### In this case, the *OptionExt* feature only contains a value of *NaN*. Therefore, we can exclude it in our data.

In [3]:
options['OptionExt'].unique()

array([nan])

In [4]:
options['Exchange'].unique()

array(['*'], dtype=object)

In [5]:
options['UnderlyingPrice'].unique()

array([201.07])

In [6]:
options['Expiration'].unique()

array(['01/05/2018', '01/12/2018', '01/19/2018', '01/26/2018',
       '02/02/2018', '02/09/2018', '02/16/2018', '03/16/2018',
       '04/20/2018', '06/15/2018', '01/18/2019', '06/21/2019',
       '01/17/2020'], dtype=object)

#### The *Type* feature in our data contains strings to classify the data. We will convert the data into integers by using the <code>astype()</code> method and passing *int* as an argument.

In [7]:
options['Type'] = (options['Type'] != 'call').astype(int)

#### Now we can represent the *Type* feature as integers and drop the irrelevant features from our data.

In [8]:
X = options.drop(['DataDate', 'OptionExt', 'AKA', 'OptionSymbol', 'Exchange', 'UnderlyingSymbol'], axis=1).copy()

In [9]:
X

,UnderlyingPrice,Type,Expiration,Strike,Last,Bid,Ask,Volume,OpenInterest,IV,Delta,Gamma,Theta,Vega
0,201.07,0,01/05/2018,140.0,0.00,60.65,61.70,0,0,0.4771,1.0000,0.0000,-2.2535,0.0000
1,201.07,1,01/05/2018,140.0,0.01,0.00,0.03,33,0,0.5949,0.0000,0.0000,0.0000,0.0000
2,201.07,0,01/05/2018,145.0,0.00,55.55,56.70,0,0,0.4771,1.0000,0.0000,-2.3340,0.0000
3,201.07,1,01/05/2018,145.0,0.00,0.00,0.03,0,0,0.5949,0.0000,0.0000,0.0000,0.0000
4,201.07,0,01/05/2018,150.0,37.00,50.85,51.70,0,9,0.4771,1.0000,0.0000,-2.4145,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,201.07,1,01/17/2020,270.0,0.00,79.05,83.50,0,0,0.3462,-0.6113,0.0039,-6.0559,110.0646
1080,201.07,0,01/17/2020,280.0,19.15,17.00,21.50,6,340,0.3482,0.3627,0.0038,-10.0582,107.6948
1081,201.07,1,01/17/2020,280.0,0.00,86.50,91.00,0,0,0.3443,-0.6409,0.0038,-5.5576,107.3281
1082,201.07,0,01/17/2020,290.0,16.70,14.95,17.25,792,181,0.3361,0.3247,0.0037,-9.3033,103.3016


### **B. Explanation of Features**

&emsp; *provided by Historical Option Data: https://www.historicaloptiondata.com/content/historical-options-data-file-structures-0*
* #### **Expiration:** This is the time it takes until the stock option expires. After the contract expires, the buyer must commit to their option (call or put).
* #### **Strike:** The price at which an option holder can buy or sell.
* #### **Last:** The last traded price of the option.
* #### **Bid:** The price an option buyer is willing to pay for the contract.
* #### **Ask:** The price an option seller is willing to sell for the contract.
* #### **Volume:** The number of contracts traded.
* #### **OpenInterest:** Always a day behind - the number of options that are held by traders and investors in active positions.
* #### **Implied Volatility(IV):** A measure of the estimate of how much the price could change. It relates to how likely the traders believe the option can make a change.
* #### **Delta:** A measure of how much the option price would change in relation to the underlying stock price. Delta = .50 -> option changes 50 cents for every 1 dollar the stock moves.
* #### **Gamma:** A measure of how fast the Delta will change when the stock price changes. A high number means this is a very explosive option and can gain or lose value quickly.
* #### **Theta:** A measure of how fast the option is losing value per day due to time decay. As the expiration day arrives, the theta increases.
* #### **Vega:** A meaasure of how sensitive the option price is to a change in the implied volatility. Options that have a long time until expiration are more sensitive to change in IV.

### **C. Preprocessing the Data**

#### Here, we converted the date into the days until expiration with the <code>to_datetime()</code> method.

In [10]:
X['Expiration'] = pd.to_datetime(X['Expiration']).sub(pd.Timestamp('01/02/2018')).dt.days
X

,UnderlyingPrice,Type,Expiration,Strike,Last,Bid,Ask,Volume,OpenInterest,IV,Delta,Gamma,Theta,Vega
0,201.07,0,3,140.0,0.00,60.65,61.70,0,0,0.4771,1.0000,0.0000,-2.2535,0.0000
1,201.07,1,3,140.0,0.01,0.00,0.03,33,0,0.5949,0.0000,0.0000,0.0000,0.0000
2,201.07,0,3,145.0,0.00,55.55,56.70,0,0,0.4771,1.0000,0.0000,-2.3340,0.0000
3,201.07,1,3,145.0,0.00,0.00,0.03,0,0,0.5949,0.0000,0.0000,0.0000,0.0000
4,201.07,0,3,150.0,37.00,50.85,51.70,0,9,0.4771,1.0000,0.0000,-2.4145,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,201.07,1,745,270.0,0.00,79.05,83.50,0,0,0.3462,-0.6113,0.0039,-6.0559,110.0646
1080,201.07,0,745,280.0,19.15,17.00,21.50,6,340,0.3482,0.3627,0.0038,-10.0582,107.6948
1081,201.07,1,745,280.0,0.00,86.50,91.00,0,0,0.3443,-0.6409,0.0038,-5.5576,107.3281
1082,201.07,0,745,290.0,16.70,14.95,17.25,792,181,0.3361,0.3247,0.0037,-9.3033,103.3016


#### In order to stay consistent with the other features, we must convert the time until expiration into units of years.

In [11]:
X['Expiration'] = X['Expiration'].apply(lambda x: x / 365)
X

,UnderlyingPrice,Type,Expiration,Strike,Last,Bid,Ask,Volume,OpenInterest,IV,Delta,Gamma,Theta,Vega
0,201.07,0,0.008219,140.0,0.00,60.65,61.70,0,0,0.4771,1.0000,0.0000,-2.2535,0.0000
1,201.07,1,0.008219,140.0,0.01,0.00,0.03,33,0,0.5949,0.0000,0.0000,0.0000,0.0000
2,201.07,0,0.008219,145.0,0.00,55.55,56.70,0,0,0.4771,1.0000,0.0000,-2.3340,0.0000
3,201.07,1,0.008219,145.0,0.00,0.00,0.03,0,0,0.5949,0.0000,0.0000,0.0000,0.0000
4,201.07,0,0.008219,150.0,37.00,50.85,51.70,0,9,0.4771,1.0000,0.0000,-2.4145,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,201.07,1,2.041096,270.0,0.00,79.05,83.50,0,0,0.3462,-0.6113,0.0039,-6.0559,110.0646
1080,201.07,0,2.041096,280.0,19.15,17.00,21.50,6,340,0.3482,0.3627,0.0038,-10.0582,107.6948
1081,201.07,1,2.041096,280.0,0.00,86.50,91.00,0,0,0.3443,-0.6409,0.0038,-5.5576,107.3281
1082,201.07,0,2.041096,290.0,16.70,14.95,17.25,792,181,0.3361,0.3247,0.0037,-9.3033,103.3016


## **3. The Black-Scholes Model**

#### The Black-Scholes Model, or Black-Scholes-Merton (BSM) Model, is one of the most well-known models used in finance. The model implies that the most relevant features of pricing a stock option depends on:
#### &emsp;&emsp;$S$: Stock Price
#### &emsp;&emsp;$X$: Exercise Price
#### &emsp;&emsp;$r$: Risk-Free Interest Rate
#### &emsp;&emsp;$T$: Time until Expiration
#### &emsp;&emsp;$\theta$: Standard Deviation of *log* returns (volatility)<br><br>

<center><h4>The BSM involves a mathematical formula that determines a fair price for stock options.</h4></center><br>

<center><h4>$C_0$ $=$ $S_0N(d_1) - Xe^{(-rt)}N(d_2)$</h4></center><br>

<center><h4>where</h4></center><br>

<center><h4>$d_1 = \frac{ln\frac{S_t}{X}+(r+\frac{\theta^2}{2}T}{\theta\sqrt{T}}$, $d_2 = \frac{ln\frac{S_t}{X}+(r-\frac{\theta^2}{2}T}{\theta\sqrt{T}}$</h4></center><br>

<center><h4>$N(X)$ gives us the probability where our random variable will be less than or equal to $X$.</h4></center><br>

<center><h4>$0 < N(X) < 1$</h4></center><br>

<center><h4>The price of the stock, $S_0$, is weighted by the probability $N(X)$:</h4></center><br>

<center><h4>$S_0N(d_1)$</h4></center><br>

<center><h4>The amount you would pay is determined by multiplying $X$ and $e$ raised to $(-rt)$, then weighted by $N(X)$:</h4></center><br>

<center><h4>$Xe^{(-rt)}N(d_2)$</h4></center>

### **A. Implementation**

#### This implementation of the BSM was provided by QuantConnect.

In [12]:
from math import log, sqrt, exp, pi
from scipy.stats import norm

In [13]:
class BsmModel:
    
    def __init__(self, option_type, price, strike, interest_rate, expiry, volatility, dividend_yield=0):
        self.type = option_type # 0 for call, 1 for put
        self.s = price # underlying asset price
        self.k = strike # strike price of option
        self.r = interest_rate # risk-free interest rate
        self.T = expiry # time till expiration in years
        self.sigma = volatility # standard deviation of log returns
        self.q = dividend_yield # dividend continuous rate
        
    def n(self, d):
        # cumulative probabilty distribution function of standard normal distribution; N(X)
        return norm.cdf(d)
    
    def dn(self, d):
        # first order derivative of n(d)
        return norm.pdf(d)
    
    def d1(self):
        d1 = (log(self.s / self.k) + (self.r - self.q + self.sigma ** 2 * 0.5) * self.T) / (self.sigma * sqrt(self.T))
        return d1
    
    def d2(self):
        d2 = (log(self.s / self.k) + (self.r - self.q - self.sigma ** 2 * 0.5) * self.T) / (self.sigma * sqrt(self.T))
        return d2
    
    def bsm_price(self):
        d1 = self.d1()
        d2 = d1 - self.sigma * sqrt(self.T)
        if self.type == 0:
            price = exp(-self.r*self.T) * (self.s * exp((self.r - self.q)*self.T) * self.n(d1) - self.k * self.n(d2))
            return price
        elif self.type == 1:
            price = exp(-self.r*self.T) * (self.k * self.n(-d2) - (self.s * exp((self.r - self.q)*self.T) * self.n(-d1)))
            return price
        else:
            print("option either 0 (call) or 1 (put)")

In [14]:
bsmX = X[['Type', 'UnderlyingPrice', 'Strike', 'OpenInterest', 'Expiration', 'IV']]
bsmX

,Type,UnderlyingPrice,Strike,OpenInterest,Expiration,IV
0,0,201.07,140.0,0,0.008219,0.4771
1,1,201.07,140.0,0,0.008219,0.5949
2,0,201.07,145.0,0,0.008219,0.4771
3,1,201.07,145.0,0,0.008219,0.5949
4,0,201.07,150.0,9,0.008219,0.4771
...,...,...,...,...,...,...
1079,1,201.07,270.0,0,2.041096,0.3462
1080,0,201.07,280.0,340,2.041096,0.3482
1081,1,201.07,280.0,0,2.041096,0.3443
1082,0,201.07,290.0,181,2.041096,0.3361


In [15]:
a = []
for i in range(len(bsmX)):
    temp = BsmModel(bsmX.iloc[i, 0], bsmX.iloc[i, 1], bsmX.iloc[i, 2], bsmX.iloc[i, 3], bsmX.iloc[i, 4], bsmX.iloc[i, 5])
    try:
        a.append(temp.bsm_price())
    except OverflowError:
        temp = float('inf')
        a.append(temp)
bsmX['BSM'] = a

D:\Users\mdhac\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [16]:
bsmX

,Type,UnderlyingPrice,Strike,OpenInterest,Expiration,IV,BSM
0,0,201.07,140.0,0,0.008219,0.4771,6.107000e+01
1,1,201.07,140.0,0,0.008219,0.5949,1.240181e-11
2,0,201.07,145.0,0,0.008219,0.4771,5.607000e+01
3,1,201.07,145.0,0,0.008219,0.5949,9.744871e-10
4,0,201.07,150.0,9,0.008219,0.4771,6.176543e+01
...,...,...,...,...,...,...,...
1079,1,201.07,270.0,0,2.041096,0.3462,8.817778e+01
1080,0,201.07,280.0,340,2.041096,0.3482,2.010700e+02
1081,1,201.07,280.0,0,2.041096,0.3443,9.608468e+01
1082,0,201.07,290.0,181,2.041096,0.3361,2.010700e+02


## **4. The Greek Letters**

#### In our dataset, there are 4 greek letters that were provided: *Delta, Gamma, Theta*, and *Vega*. Each greek letter is a parameter that is used to measure different dimensions in options.
* #### Delta: The measure of how much the option price would change in relation to the underlying stock price.
* #### Gamma: The measure of how fast the Delta will change when the stock prices change.
* #### Theta: The measure of how fast the option is losing value per day due to time decay.
* #### Vega: The measure of how sensitive the option price is to a change in the implied volatility.
* #### Rho: The measure of how sensitive the option price is to a change in interest rate.

#### These values are typically calculated on a daily basis. Once the values for these letters have been determined, traders would apply them to the BSM model to asses option prices.

### **A. Implementation**

#### Here, we will add methods involving the greek letters to the BsmModel class. Since our dataset does not provide the parameter for Rho, we will not be implementing a method for it.

In [19]:
from functools import wraps

def add_method(cls):
    def decorator(func):
        @wraps(func) 
        def wrapper(self, *args, **kwargs): 
            return func(*args, **kwargs)
        setattr(cls, func.__name__, wrapper)
        # Note we are not binding func, but wrapper which accepts self but does exactly the same as func
        return func # returning func means func can still be used normally
    return decorator

In [20]:
@add_method(BsmModel)
def delta(self):
    d1 = self.d1()
    if self.type == 0:
        return exp(-self.q * self.T) * self.n(d1)
    elif self.type == 1:
        return exp(-self.q * self.T) * (self.n(d1)-1)

@add_method(BsmModel)
def gamma(self, ):
    d1 = self.d1()
    dn1 = self.dn(d1)
    return dn1 * exp(-self.q * self.T) / (self.s * self.sigma * sqrt(self.T))
    
@add_method(BsmModel)
def theta(self):
    d1 = self.d1()
    d2 = d1 - self.sigma * sqrt(self.T)
    dn1 = self.dn(d1)

    if self.type == 0:
        theta = -self.s * dn1 * self.sigma * exp(-self.q*self.T) / (2 * sqrt(self.T)) \
                + self.q * self.s * self.n(d1) * exp(-self.q*self.T) \
                - self.r * self.k * exp(-self.r*self.T) * self.n(d2)
        return theta
    elif self.type == 1:
        theta = -self.s * dn1 * self.sigma * exp(-self.q * self.T) / (2 * sqrt(self.T)) \
                - self.q * self.s * self.n(-d1) * exp(-self.q * self.T) \
                + self.r * self.k * exp(-self.r * self.T) * self.n(-d2)
        return theta
        
@add_method(BsmModel)
def vega(self):
    d1 = self.d1()
    dn1 = self.dn(d1)
    return self.s * sqrt(self.T) * dn1 * exp(-self.q * self.T)

In [22]:
greekX = X[['Type', 'UnderlyingPrice', 'Strike', 'OpenInterest', 'Expiration', 'IV', 'Delta', 'Gamma', 'Theta', 'Vega']]
greekX

,Type,UnderlyingPrice,Strike,OpenInterest,Expiration,IV,Delta,Gamma,Theta,Vega
0,0,201.07,140.0,0,0.008219,0.4771,1.0000,0.0000,-2.2535,0.0000
1,1,201.07,140.0,0,0.008219,0.5949,0.0000,0.0000,0.0000,0.0000
2,0,201.07,145.0,0,0.008219,0.4771,1.0000,0.0000,-2.3340,0.0000
3,1,201.07,145.0,0,0.008219,0.5949,0.0000,0.0000,0.0000,0.0000
4,0,201.07,150.0,9,0.008219,0.4771,1.0000,0.0000,-2.4145,0.0000
...,...,...,...,...,...,...,...,...,...,...
1079,1,201.07,270.0,0,2.041096,0.3462,-0.6113,0.0039,-6.0559,110.0646
1080,0,201.07,280.0,340,2.041096,0.3482,0.3627,0.0038,-10.0582,107.6948
1081,1,201.07,280.0,0,2.041096,0.3443,-0.6409,0.0038,-5.5576,107.3281
1082,0,201.07,290.0,181,2.041096,0.3361,0.3247,0.0037,-9.3033,103.3016
